In [156]:
import pandas as pd
import math
from decimal import Decimal
from constants import MONGO_URL_DATA_FILE_PATH, MONGO_URL_COLUMNS, AUTHOR_DATA_FILE_PATH
MONGO_URL_DATA_FILE_PATH

'/home/tom/codewithcheese/steem/notebooks/./data/mongo_url_list.csv'

In [2]:
url_data = pd.DataFrame.from_csv(MONGO_URL_DATA_FILE_PATH)

In [3]:
url_data.head()

,id,url,host_name,domain,author,parent_author,parent_permlink,permlink
0,4,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1
1,19,https://github.com/wackou,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
2,19,https://github.com/wackou/bts_tools,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
3,19,https://bts-tools.readthedocs.io/,bts-tools.readthedocs.io,readthedocs.io,wackou,NaN,witness-category,wackou-witness-post
4,19,https://steemit.com/steem/@wackou/btstools-rel...,steemit.com,steemit.com,wackou,NaN,witness-category,wackou-witness-post


In [4]:
author_data = pd.DataFrame.from_csv(AUTHOR_DATA_FILE_PATH)

Calculate reputation score
https://steemit.com/steemit/@digitalnotvir/how-reputation-scores-are-calculated-the-details-explained-with-simple-math

In [189]:
def reputation_score(reputation):
    neg = False if reputation >= 0 else True
    out = math.log(math.fabs(reputation), 10)
    out = max(out - 9, 0)
    out = out * -1 if neg else out
    out = (out * 9) + 25
    return math.floor(out)

In [190]:
author_data.iloc[39]

name                      bittrexrichie
created             2016-04-18 00:04:24
last_vote_time      2016-04-24 21:36:30
last_post           2016-04-22 06:44:21
reputation                -463779708716
total_steem                           0
total_sbd                             0
total_vests                           0
reputation_score                1.00319
Name: 39, dtype: object

In [191]:
reputation_score(author_data.iloc[39]['reputation'])

1

In [192]:
author_data['reputation_score'] = author_data.apply(lambda row: reputation_score(row['reputation']), axis=1)

In [193]:
author_data.head()

,name,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
1,wackou,2016-04-02 13:23:15,2017-09-30 11:56:51,2017-09-09 21:28:36,5227870101994,6213.431,10.220,1.112955e+09,58
2,xeroc,2016-04-08 13:48:54,2017-09-29 12:16:06,2017-09-22 13:19:15,116133494848670,0.640,289.838,5.189552e+07,70
3,roadscape,2016-04-06 01:03:54,2017-09-29 03:04:36,2017-09-28 21:07:39,13365094119608,52.273,3217.628,1.948232e+09,62
4,fmooo,2016-03-25 01:08:18,2017-07-26 15:31:48,2016-04-20 14:38:06,10062655,0.000,0.001,2.403510e+05,25


In [194]:
url_with_author_data = pd.merge(url_data, author_data, left_on='author', right_on='name')
url_with_author_data.head()

,id,url,host_name,domain,author,parent_author,parent_permlink,permlink,name,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,4,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
1,96,http://www.washingtonpost.com/wp-srv/national/...,www.washingtonpost.com,washingtonpost.com,red,NaN,spam,spamtford,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
2,97,https://www.youtube.com/watch?v=B0N1gQXETmU,www.youtube.com,youtube.com,red,NaN,Does voting even matter?,newswearechange,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
3,140,http://blog.tenthamendmentcenter.com/2016/04/s...,blog.tenthamendmentcenter.com,tenthamendmentcenter.com,red,NaN,news,governor-mississippi-constitutional-carry-law-...,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
4,142,https://scontent-ord1-1.xx.fbcdn.net/hphotos-x...,scontent-ord1-1.xx.fbcdn.net,fbcdn.net,red,NaN,news,laughable,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54


### Most mentioned urls

In [195]:
url_data['url'].value_counts().head()

https://steemit.com/steemhelp/@dele-puppy/register-bot          8
https://bitshares.openledger.info/?r=infobot#/create-account    6
https://steemit.com/                                            6
http://www.bbc.com/news/technology-36168863                     4
http://gavinandresen.ninja/satoshi                              4
Name: url, dtype: int64

### Top 20 websites by total mentions



In [196]:
url_data['domain'].value_counts()[0:20]

imgur.com            154
youtube.com          124
steemit.com           94
github.com            76
wordpress.com         56
youtu.be              42
soundcloud.com        36
imgsafe.org           36
reddit.com            31
openledger.info       26
bitsharestalk.org     25
twitter.com           24
postimg.org           21
staticflickr.com      20
bitcointalk.org       20
google.com            20
blogspot.com          17
wikipedia.org         15
cryptofresh.com       14
vidble.com            13
Name: domain, dtype: int64

### Top 20 websites by number of unique authors

In [197]:
domain_authors = url_with_author_data.groupby('domain', as_index=False)['author'].agg({'authors': lambda x: pd.unique(x), 'author_count': lambda x: len(pd.unique(x))})
domain_authors.sort_values(by='author_count', ascending=False).head(n=20)

,domain,authors,author_count
140,imgur.com,"[fmooo, steem-id, penambang, kushed, dele-pupp...",35
341,youtube.com,"[red, steem-id, samupaha, ihashfury, coin, enk...",31
117,github.com,"[wackou, xeroc, fmooo, steempty, nextgencrypto...",30
270,steemit.com,"[wackou, xeroc, steempty, nextgencrypto, phare...",29
235,reddit.com,"[jabbasteem2, arhag, tuck-fheman, steemit-news...",12
37,bitsharestalk.org,"[xeroc, nextgencrypto, xeldal, clayop, witness...",12
306,twitter.com,"[joseph, tuck-fheman, masteryoda, jamesstroud,...",10
123,google.com,"[mrs.agsexplorer, au1nethyb1, tuck-fheman, ste...",10
29,bitcointalk.org,"[kushed, steemed, arhag, tuck-fheman, stoner19...",10
328,wikipedia.org,"[abit, steemed, dele-puppy, dantheman, arhag, ...",9


In [198]:
url_data.groupby(['domain'])['author'].nunique().sort_values(ascending=False)[0:20]

domain
imgur.com            35
youtube.com          31
github.com           30
steemit.com          29
bitsharestalk.org    12
reddit.com           12
google.com           10
bitcointalk.org      10
twitter.com          10
openledger.info       9
wikipedia.org         9
postimg.org           9
youtu.be              9
imgsafe.org           6
ipfs.pics             6
cryptofresh.com       6
steemd.com            5
blogspot.com          5
facebook.com          5
slack.com             4
Name: author, dtype: int64

### Top 20 websites by author reputation

Join url data with author data so that we can group by websites, unique on author and sum on reputation

Websites with highest average reputation

In [201]:
domain_mean_reputation = url_with_author_data.groupby('domain', as_index=False)['author'].agg({'reputation_mean':(lambda x: author_data[author_data['name'].isin(pd.unique(x))]['reputation_score'].mean())}).sort_values(by='reputation_mean', ascending=False)
domain_rep_with_authors = pd.merge(domain_mean_reputation, domain_authors)
domain_rep_with_authors.head(n=10)

,domain,reputation_mean,authors,author_count
0,themacro.com,73.0,tuck-fheman,1
1,globalresearch.ca,73.0,tuck-fheman,1
2,voxelsoft.com,73.0,dantheman,1
3,ddees.com,73.0,tuck-fheman,1
4,bloomberg.com,73.0,tuck-fheman,1
5,techcrunch.com,73.0,"[tuck-fheman, donkeypong]",2
6,gortool.com,73.0,tuck-fheman,1
7,theantimedia.org,73.0,tuck-fheman,1
8,visualcapitalist.com,73.0,tuck-fheman,1
9,goo.gl,73.0,tuck-fheman,1


Websites with higest average reputation that have 10 more mentions

In [202]:
domain_rep_with_authors[domain_rep_with_authors.author_count >= 10].head(n=10)

,domain,reputation_mean,authors,author_count
194,steemit.com,62.344828,"[wackou, xeroc, steempty, nextgencrypto, phare...",29
196,twitter.com,62.200000,"[joseph, tuck-fheman, masteryoda, jamesstroud,...",10
197,bitsharestalk.org,62.166667,"[xeroc, nextgencrypto, xeldal, clayop, witness...",12
207,bitcointalk.org,61.200000,"[kushed, steemed, arhag, tuck-fheman, stoner19...",10
228,github.com,59.100000,"[wackou, xeroc, fmooo, steempty, nextgencrypto...",30
261,youtube.com,56.806452,"[red, steem-id, samupaha, ihashfury, coin, enk...",31
267,google.com,55.500000,"[mrs.agsexplorer, au1nethyb1, tuck-fheman, ste...",10
291,imgur.com,52.171429,"[fmooo, steem-id, penambang, kushed, dele-pupp...",35
300,reddit.com,50.416667,"[jabbasteem2, arhag, tuck-fheman, steemit-news...",12


Websites with highest total reputation

In [204]:
domain_mean_reputation = url_with_author_data.groupby('domain', as_index=False)['author'].agg({'reputation_mean':(lambda x: author_data[author_data['name'].isin(pd.unique(x))]['reputation_score'].sum())})
domain_mean_reputation.sort_values(by='reputation_mean', ascending=False).head(n=20)

,domain,reputation_mean
140,imgur.com,1826
270,steemit.com,1808
117,github.com,1773
341,youtube.com,1761
37,bitsharestalk.org,746
306,twitter.com,622
29,bitcointalk.org,612
235,reddit.com,605
328,wikipedia.org,592
123,google.com,555


In [205]:
url_with_author_data.loc[url_with_author_data.domain=='tradingview.com']

,id,url,host_name,domain,author,parent_author,parent_permlink,permlink,name,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
991,1124,https://www.tradingview.com/x/gE07TWCp/,www.tradingview.com,tradingview.com,timifey2015,NaN,eth,eth-cryptocurrency-analytics,timifey2015,2016-04-30 12:14:45,2016-08-21 10:02:54,2016-07-22 10:52:06,-193140276839,39.415,0.419,60138.721,4
992,1125,https://www.tradingview.com/x/zn1HKbDl/,www.tradingview.com,tradingview.com,timifey2015,NaN,btc,btc-cryptocurrency-analytics,timifey2015,2016-04-30 12:14:45,2016-08-21 10:02:54,2016-07-22 10:52:06,-193140276839,39.415,0.419,60138.721,4
993,1129,https://www.tradingview.com/x/KvaYhBAf/,www.tradingview.com,tradingview.com,timifey2015,NaN,nem,nem-cryptocurrency-analytics,timifey2015,2016-04-30 12:14:45,2016-08-21 10:02:54,2016-07-22 10:52:06,-193140276839,39.415,0.419,60138.721,4
994,1819,https://www.tradingview.com/x/eMN4GL70/,www.tradingview.com,tradingview.com,timifey2015,NaN,ethereum,trade-ethereum,timifey2015,2016-04-30 12:14:45,2016-08-21 10:02:54,2016-07-22 10:52:06,-193140276839,39.415,0.419,60138.721,4


### Top 20 websites by author steem power

### Top 20 websites by log of author steem power